In [ ]:
!pip install --upgrade tensorflow==1.15.0

In [ ]:
!pip install --upgrade systemml

In [ ]:
import systemml

In [ ]:
import tensorflow
print(tensorflow.__version__)

Import all dependencies 

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from tensorflow.keras.layers import Dense, Input, Dropout, LSTM, Activation
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.metrics import mean_squared_error, mean_absolute_error
from tensorflow.keras.callbacks import Callback

from tensorflow import keras

In [ ]:
import numpy as np
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
import sklearn
from sklearn.preprocessing import MinMaxScaler
#from sklearn.metrics import mean_squared_error
#from keras.models import Sequential
#from keras.layers import Dense, Dropout
#from keras.layers import LSTM
#from keras.callbacks import Callback
#from keras.models import Sequential
#from keras.layers import LSTM, Dense, Activation
import pickle
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import sys
from queue import Queue
import pandas as pd
import json
%matplotlib inline

## Load data

Files for training are sampled from the lorenz attractor model implemented in NodeRED. https://developer.ibm.com/tutorials/iot-deep-learning-anomaly-detection-2/

In [ ]:
#!rm watsoniotp.*
!wget https://raw.githubusercontent.com/romeokienzler/developerWorks/master/lorenzattractor/watsoniotp.healthy.phase_aligned.pickle
!wget https://raw.githubusercontent.com/romeokienzler/developerWorks/master/lorenzattractor/watsoniotp.broken.phase_aligned.pickle

In [ ]:
!mv watsoniotp.healthy.phase_aligned.pickle watsoniotp.healthy.pickle
!mv watsoniotp.broken.phase_aligned.pickle watsoniotp.broken.pickle

In [ ]:
data_healthy = pickle.load(open('watsoniotp.healthy.pickle', 'rb'), encoding='latin1')
data_broken = pickle.load(open('watsoniotp.broken.pickle', 'rb'), encoding='latin1')

In [ ]:
data_healthy.shape

> 3 vibration sensor axes and 3000 samples

# I) Anomaly detection with supervised classification

> Anomaly detection of data from **Lorenz attractor model**

Assignment from IBM's Coursera course **Applied AI with DeepLearning** as part of the [**Advanced Data Science Specialization**](https://www.coursera.org/specializations/advanced-data-science-ibm#courses).

Can construct a simple feed forward neural net, 3 dense layers, trained on healthy and broken samples.
If we were to run into a problem of unsupervised anomaly detection, best bet would be to use an autoencoder and train it with an LSTM on what we believe to be non-anonmalous data.


Import keras tuner to optimize model hyperparameters

In [ ]:
!pip install keras-tuner

## 1 - Visualize data 

In [ ]:
fig = plt.figure()
ax = fig.gca(projection='3d')

ax.plot(data_healthy[:,0], data_healthy[:,1], data_healthy[:,2],lw=0.5)
ax.set_xlabel("X Axis")
ax.set_ylabel("Y Axis")
ax.set_zlabel("Z Axis")
ax.set_title("Lorenz Attractor for Healthy Data");

In [ ]:
data_healthy_df = pd.DataFrame(data_healthy)

data_healthy_df.head()

Then for the broken one

In [ ]:
fig = plt.figure()
ax = fig.gca(projection='3d')

ax.plot(data_broken[:,0], data_broken[:,1], data_broken[:,2],lw=0.5)
ax.set_xlabel("X Axis")
ax.set_ylabel("Y Axis")
ax.set_zlabel("Z Axis")
ax.set_title("Lorenz Attractor for Broken Data");

## 2 - Transform data

### Fast Fourier Transform

In [ ]:
data_healthy_fft = np.fft.fft(data_healthy).real # sine decomposition
data_broken_fft = np.fft.fft(data_broken).real 

In [ ]:
print(data_healthy_fft.shape)

In [ ]:
fig, ax = plt.subplots(num=None, figsize=(14, 6), dpi=80, facecolor='w', edgecolor='k')
size = len(data_healthy_fft)
ax.plot(range(0,size), data_healthy_fft[:,0].real, '-', color=(0,0.5,1), animated = True, linewidth=1, label=['sample 1'])
ax.plot(range(0,size), data_healthy_fft[:,1].real, '-', color=(1,0,0.5), animated = True, linewidth=1, label=['sample 2'])
ax.plot(range(0,size), data_healthy_fft[:,2].real, '-', color=(1,0.5,0), animated = True, linewidth=1, label=['sample 3'])
ax.set_xlabel('Frequency')
ax.set_ylabel('Magnitude')
ax.set_title('Frequency spectrum for healthy data')
ax.legend()
None

> Samples 2 and 3 (second and third axes) are overlayed

In [ ]:
fig, ax = plt.subplots(num=None, figsize=(14, 6), dpi=80, facecolor='w', edgecolor='k')
size = len(data_healthy_fft)
ax.plot(range(0,size), data_broken_fft[:,0].real, '-', color=(0,0.5,1), animated = True, linewidth=1, label=['sample 1'])
ax.plot(range(0,size), data_broken_fft[:,1].real, '-', color=(1,0,0.5), animated = True, linewidth=1, label=['sample 2'])
ax.plot(range(0,size), data_broken_fft[:,2].real, '-', color=(1,0.5,0), animated = True, linewidth=1, label=['sample 3'])
ax.set_xlabel('Frequency')
ax.set_ylabel('Magnitude')
ax.set_title('Frequency spectrum for broken data')
ax.legend()
None

## 3 - Scale data for neural network

In [ ]:
def scaleData(data):
    scaler = MinMaxScaler(feature_range=(0, 1))
    return scaler.fit_transform(data)

In [ ]:
data_healthy_scaled = scaleData(data_healthy_fft)
data_broken_scaled = scaleData(data_broken_fft)

Reshape to have three samples (rows, training example) and 3000 features (columns, features). 

In [ ]:
data_healthy_scaled = data_healthy_scaled.T
data_broken_scaled = data_broken_scaled.T

In [ ]:
data_broken_scaled.shape

In [ ]:
#data_healthy_scaled.reshape(3, 3000)
#data_broken_scaled.reshape(3, 3000)

## 4 - Model

In [ ]:
# callback to keep track of losses for keras

class LossHistory(Callback):
    def on_train_begin(self, logs={}):
        self.losses = []

    def on_batch_end(self, batch, logs={}):
        sys.stdout.write(str(logs.get('loss'))+str(', '))
        sys.stdout.flush()
        self.losses.append(logs.get('loss'))
        
lr = LossHistory()

### use kerastuner for hyperparameter tuning

In [ ]:
from kerastuner import HyperModel
from tensorflow.keras import optimizers

class FFHyperModel(HyperModel):
    def __init__(self, input_shape, num_classes):
        self.input_shape = input_shape
        self.num_classes = num_classes
    
    def build(self, hp):
        model = Sequential()
        model.add(
            Dense(
                units=hp.Int(
                    'units_l1',
                    min_value=32,
                    max_value=480,
                    step=32,
                    default=192
                ),
                input_shape=self.input_shape,
                activation='relu'
            )
        )
        model.add(
            Dense(
                units=hp.Int(
                    'units_l2',
                    min_value=32,
                    max_value=480,
                    step=32,
                    default=96
                ),
                activation='relu'
            )        
        )
        model.add(Dense(self.num_classes, activation='sigmoid'))
        
        model.compile(
            optimizer=optimizers.SGD(
                hp.Float(
                    'learning_rate',
                    min_value=1e-4,
                    max_value=1e-1,
                    sampling='LOG',
                    default=1e-2
                ),
                clipnorm=1.),
            loss='binary_crossentropy',
            metrics=['accuracy']
        )
        
        return model
                  

In [ ]:
dim = 3000
samples = 3
num_classes = 1

In [ ]:
# supervised, have to add labels (0-broken, 1-healthy) to training data before we train

label_healthy = np.repeat(1,3)
label_healthy.shape = (3,1)
label_broken = np.repeat(0,3)
label_broken.shape = (3,1)

train_healthy = np.hstack((data_healthy_scaled,label_healthy))
train_broken = np.hstack((data_broken_scaled,label_broken))
train_both = np.vstack((train_healthy,train_broken))

In [ ]:
hypermodel= FFHyperModel(input_shape=(dim,), num_classes=num_classes)

In [ ]:
hypermodel.input_shape

In [ ]:
from kerastuner.tuners import Hyperband

# Hyperband : optimized version of random search, uses early stopping

epochs=2500
tuner=Hyperband(hypermodel, max_epochs=epochs, objective='loss', executions_per_trial=2, 
                  directory='hyperband', project_name='supervised_anomaly')

In [ ]:
tuner.search_space_summary()

In [ ]:
features = train_both[:,:3000] 
labels = train_both[:,3000:] 

In [ ]:
# same as model.fit
epochs = 2500

tuner.search(features, labels, epochs=epochs)

In [ ]:
tuner.results_summary()

In [ ]:
best_model = tuner.get_best_models(num_models=1)[0]

In [ ]:
best_model.summary()

In [ ]:
loss, accuracy = best_model.evaluate(features, labels)

In [ ]:
print(f'Loss: {loss}, Accuracy: {accuracy}')

In [ ]:
pred_healthy = best_model.predict(data_healthy_scaled)
pred_broken = best_model.predict(data_healthy_scaled)

In [ ]:
pred_healthy

In [ ]:
pred_broken

### without the tuner

In [ ]:
number_of_neurons_layer1 = 200
number_of_neurons_layer2 = 100
number_of_neurons_layer3 = 1 #output: 0,1
number_of_epochs = 2500

In [ ]:
from tensorflow.keras import optimizers
sgd = optimizers.SGD(lr=0.01, clipnorm=1.)

model = Sequential()
model.add(Dense(number_of_neurons_layer1, input_shape=(dim, ), activation='relu'))
model.add(Dense(number_of_neurons_layer2, activation='relu'))
model.add(Dense(number_of_neurons_layer3, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer=sgd)

def train(data,label): 
    model.fit(data, label, epochs=number_of_epochs, batch_size=72, validation_data=(data, label), verbose=False, shuffle=True, callbacks=[lr])
    
    # ideally, would split data for validation set, but since we only have three examples of each class limited options

def score(data):
    return model.predict(data)

In [ ]:
train(features,labels)

In [ ]:
fig, ax = plt.subplots(num=None, figsize=(14, 6), dpi=80, facecolor='w', edgecolor='k')
size = len(lr.losses)
ax.plot(range(0,size), lr.losses, '-', color=(0,0.5,1), animated = True, linewidth=1)
ax.set_ylabel('Loss, binary cross entropy')
ax.set_xlabel('Epochs');

In [ ]:
fig, ax = plt.subplots(num=None, figsize=(14, 6), dpi=80, facecolor='w', edgecolor='k')
size = len(lr.losses)
ax.plot(range(0,size), lr.losses, '-', color=(0,0.5,1), animated = True, linewidth=1)
ax.set_ylabel('Loss, binary cross entropy')
ax.set_xlabel('Epochs')
ax.set_ylim(0,0.01);

In [ ]:
# ideally would split for test data set as well

score(data_healthy_scaled)

In [ ]:
score(data_broken_scaled)

# Anomaly detection with unsupervised classification
Using an LSTM autoencoder trained on healthy data, and setting an error threshold above which the data will be classified as an anomaly (i.e. can't be reconstructed with the autoencoder)

In [ ]:
# healthy sensor data 

fig, ax = plt.subplots(num=None, figsize=(14, 6), dpi=80, facecolor='w', edgecolor='k')
size = len(data_healthy)
ax.plot(range(0,size), data_healthy[:,0], '-', color=(0,0.5,1), linewidth=1, label='sensor 1')
ax.plot(range(0,size), data_healthy[:,1], '-', color=(1,0,0.5), linewidth=1, label='sensor 2')
ax.plot(range(0,size), data_healthy[:,2], '-', color=(1,0.5,0), linewidth=1, label='sensor 3')
#ax.set_xlabel('Frequency')
#ax.set_ylabel('Magnitude')
ax.set_title('Healthy data')
ax.legend()
None

In [ ]:
# frequency domain 

fig, ax = plt.subplots(num=None, figsize=(14, 6), dpi=80, facecolor='w', edgecolor='k')
size = len(data_healthy_fft)
ax.plot(range(0,size), data_healthy_fft[:,0].real, '-', color=(0,0.5,1), linewidth=1, label='X')
ax.plot(range(0,size), data_healthy_fft[:,1].real, '-', color=(1,0,0.5), linewidth=1, label='Y')
ax.plot(range(0,size), data_healthy_fft[:,2].real, '-', color=(1,0.5,0), linewidth=1, label='Z')
ax.set_xlabel('Frequency')
ax.set_ylabel('Magnitude')
ax.set_title('Frequency spectrum for healthy data')
ax.legend()
None

Make sure data is scaled (MinMax scaler was run for part I) and reshape for LSTM.

> LSTM cells expect a 3 dimensional tensor of the form [data samples, time steps, features]. Here, each sample input into the LSTM network represents one step in time and contains 3 features — the sensor readings for the three axes at that time step.

In [ ]:
data_healthy.shape[0]

In [ ]:
data_scaled = scaleData(data_healthy)
data_shaped = data_scaled.reshape(data_scaled.shape[0], 1, data_scaled.shape[1])

In [ ]:
data_shaped.shape

In [ ]:
data_broken_scaled = scaleData(data_broken)
data_broken_shaped = data_broken_scaled.reshape(data_broken_scaled.shape[0], 1, data_broken_scaled.shape[1])

In [ ]:
units_l1, units_l2 = 16, 4

In [ ]:
# use functional Keras API
# set return_sequences to True to stack LSTM layers (except for repeat vector)

from tensorflow.keras.layers import TimeDistributed, RepeatVector

inputs = Input(
    shape=(
        data_shaped.shape[1], data_shaped.shape[2]
    )
)

layer_1 = LSTM(
    units_l1, 
    activation='relu', 
    return_sequences=True
)(inputs)

layer_2 = LSTM(
    units_l2,
    activation='relu',
    return_sequences=False
)(layer_1)

rep = RepeatVector(
    data_shaped.shape[1])(layer_2)

layer_3 = LSTM(
    units_l2,
    activation='relu',
    return_sequences=True
)(rep)

layer_4 = LSTM(
    units_l1,
    activation='relu',
    return_sequences=True
)(layer_3)

output = TimeDistributed(
    Dense(data_shaped.shape[2])
)(layer_4)

model = Model(inputs=inputs, outputs=output)

In [ ]:
model.compile(optimizer='adam', loss='mae')

In [ ]:
model.summary()

In [ ]:
EPOCHS=250
BATCH_SIZE=72

model.fit(data_shaped, data_shaped, epochs=EPOCHS, batch_size=BATCH_SIZE, callbacks=[lr])
    

In [ ]:
fig, ax = plt.subplots(num=None, figsize=(14, 6), dpi=80, facecolor='w', edgecolor='k')
size = len(lr.losses)
ax.plot(range(0,size), lr.losses, '-', color=(0,0.5,1), animated = True, linewidth=1)
ax.set_ylabel('Loss, binary cross entropy')
ax.set_xlabel('Epochs');

In [ ]:
pred = model.predict(data_shaped)

# reshape to 3000,3
pred = pred.reshape(pred.shape[0], pred.shape[2])

# compute loss
loss_mae = np.mean(np.abs(pred - data_scaled), axis=1)

In [ ]:
import seaborn as sns

sns.distplot(loss_mae, bins=30);

> use 0.027 as threshold

In [ ]:
# test with broken data

pred_broken = model.predict(data_broken_shaped)
pred_broken = pred_broken.reshape(pred_broken.shape[0], pred_broken.shape[2])

# compute loss
loss_mae_broken = np.mean(np.abs(pred_broken - data_broken_scaled), axis=1)

In [ ]:
threshold = 0.027

anomaly = loss_mae_broken > threshold
anomaly.sum()

In [ ]:
plt.figure(figsize=(16,4))
plt.plot(loss_mae)
plt.hlines(threshold, 0, len(loss_mae_broken), color='r')
plt.title('Healthy data')
plt.ylabel('mae losses');

In [ ]:
anomaly_toplot = anomaly[anomaly==1]*threshold

plt.figure(figsize=(16,4))
plt.plot(loss_mae_broken, zorder=0)
plt.hlines(threshold, 0, len(loss_mae_broken), color='r', label='threshold')
plt.scatter(np.where(anomaly==1), anomaly_toplot, color='r', s=10, label='flagged anomalies')
plt.title('Broken data, flagged as anomaly')
plt.legend()
plt.ylabel('mae losses');

## Using Apache SystemML

Can train a Keras model on Apache Spark with SystemML. SystemML runs on top of Apache Spark, and it can import keras model definitions.

In [ ]:
from systemml.mllearn import Keras2DML

EPOCHS=250
BATCH_SIZE=72

max_iter = int(EPOCHS*math.ceil(samples/BATCH_SIZE))

model_sysml = Keras2DML(
    spark, 
    model, 
    input_shape=(data_shaped.shape[1], data_shaped.shape[2]), 
    batch_size=BATCH_SIZE, 
    max_iter=max_iter, 
    test_interval=0, 
    display=10)

model_sysml.set(perform_one_hot_encoding=False)
model_sysml.set(debug=True)